In [7]:
import mne
import numpy as np
import scipy.io
import os
import sys

In [10]:
# --- Set up Project Root ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
# Add 'src' directory to Python path to import config
sys.path.append(os.path.join(PROJECT_ROOT, 'src'))

In [11]:
# --- Import configurations from config.py ---
# We'll try to import config, but this notebook is for verification so it might still work if config is slightly off.
try:
    from eeg.config import (
        RAW_MATLAB_DATA_DIR, 
        VEOG_CHANNEL_NAME, ACCEL_CHANNEL_NAMES, STIM_CHANNEL_NAME,
        ALL_SUBJECTS
    )
except ImportError:
    print("Warning: Could not import config.py. Proceeding with hardcoded paths/names for verification.")
    # Fallback paths/names if config.py isn't perfectly set up yet
    RAW_MATLAB_DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'matlab_eeg')
    VEOG_CHANNEL_NAME = 'VEOG' # Default assumption
    ACCEL_CHANNEL_NAMES = ['ACCEL_X', 'ACCEL_Y', 'ACCEL_Z'] # Default assumption
    STIM_CHANNEL_NAME = 'STI 014' # Default assumption
    ALL_SUBJECTS = [804, 890] # Default assumption for testing

In [12]:
# --- Parameters for the specific .mat file you want to check ---
# Pick one subject and session from your actual data
TEST_SUBJ_ID = ALL_SUBJECTS[0] # Use the first subject in your defined list
TEST_SESSION_NUM = 1 # Test session 1

In [14]:
# --- Construct the full path to the .mat file ---
# Based on your confirmed naming convention: [subjid]_[session]_PD_REST.mat
# OR, if you need to check the _REST1.mat version for any reason:
# mat_fname_base = f"{TEST_SUBJ_ID}_{TEST_SESSION_NUM}_PD_REST1"
mat_fname_base = f"{TEST_SUBJ_ID}_{TEST_SESSION_NUM}_PD_REST" 
mat_fname_full = os.path.join(RAW_MATLAB_DATA_DIR, f"{mat_fname_base}.mat")

print(f"Attempting to load MATLAB file for inspection: {mat_fname_full}")

Attempting to load MATLAB file for inspection: /Users/patriciawatanabe/Projects/NTUT25_Software/data/matlab_file/804_1_PD_REST.mat


In [15]:
try:
    # Load the .mat file with settings for common EEGLAB exports
    # mat_dtype=True preserves MATLAB data types for numbers
    # simplify_cells=True converts MATLAB cell arrays and structs into Python lists/dicts
    # squeeze_me=True removes single-dimensional entries (e.g., [1x1 struct] becomes just struct)
    mat_data = scipy.io.loadmat(mat_fname_full, simplify_cells=True, mat_dtype=True, squeeze_me=True)
    print(f"Successfully loaded {mat_fname_full}")

    # --- Step 1: Identify the main EEGLAB EEG struct ---
    eeg_struct = mat_data.get('EEG', None)
    if eeg_struct is None:
        print("\n--- Warning: 'EEG' variable not found at top level. Assuming mat_data itself is the EEG struct content. ---")
        eeg_struct = mat_data # Assume the whole mat_data is the EEG struct if no 'EEG' key
    else:
        print("\n--- Found 'EEG' key at top level. Proceeding with eeg_struct = mat_data['EEG']. ---")

    print(f"\nType of 'eeg_struct': {type(eeg_struct)}")
    if isinstance(eeg_struct, dict):
        print(f"Keys within 'eeg_struct': {eeg_struct.keys()}")
    else:
        print(" 'eeg_struct' is not a dictionary. Cannot list keys directly. Please inspect MATLAB file.")


    # --- Step 2: Extract Data Array and Sampling Rate ---
    print("\n--- Inspecting EEG.data and EEG.srate ---")
    if 'data' in eeg_struct:
        raw_data_array = eeg_struct['data']
        print(f" 'eeg_struct['data']' shape: {raw_data_array.shape}")
        print(f" 'eeg_struct['data']' first 5 samples of first channel: {raw_data_array[0, :5]}")
        print(f" 'eeg_struct['data']' type: {type(raw_data_array)}")
    else:
        print(" 'data' field not found in 'eeg_struct'.")
        raw_data_array = None

    if 'srate' in eeg_struct:
        sfreq = eeg_struct['srate']
        print(f" 'eeg_struct['srate']': {sfreq}")
        print(f" 'eeg_struct['srate']' type: {type(sfreq)}")
    else:
        print(" 'srate' field not found in 'eeg_struct'.")
        sfreq = None

# --- Step 3: Extract Channel Locations ('chanlocs') and Labels ---
    print("\n--- Inspecting EEG.chanlocs and extracting labels ---")
    ch_labels_mat = []
    if 'chanlocs' in eeg_struct:
        chanlocs = eeg_struct['chanlocs']
        print(f"Type of 'eeg_struct['chanlocs']': {type(chanlocs)}")

        # Print the first few elements or the whole structure to understand it
        if isinstance(chanlocs, list) and len(chanlocs) > 0:
            print(f"  First element of 'chanlocs' list: {chanlocs[0]}")
            if isinstance(chanlocs[0], dict) and 'labels' in chanlocs[0]:
                print(f"  Labels of first channel (from list of dicts): {chanlocs[0]['labels']}")
                # Most likely scenario for list of dicts
                ch_labels_mat = [loc['labels'] for loc in chanlocs]
            else:
                print("  'chanlocs' is a list, but elements are not dicts with 'labels'.")
        elif isinstance(chanlocs, np.ndarray) and chanlocs.ndim == 0: # Squeezed single struct
            if 'labels' in chanlocs.item(): # Access content if it's a scalar numpy object
                 ch_labels_mat = chanlocs.item()['labels']
                 if not isinstance(ch_labels_mat, list): # If it's a single string
                     ch_labels_mat = [ch_labels_mat] # Wrap in list
            else:
                print("  'chanlocs' is a numpy array (scalar), but has no 'labels' field as expected.")
        elif isinstance(chanlocs, dict) and 'labels' in chanlocs:
            # This handles cases where 'chanlocs' itself is a dict and 'labels' is a list/ndarray within it
            ch_labels_mat = chanlocs['labels']
            if isinstance(ch_labels_mat, np.ndarray): # Convert numpy array of strings to list
                ch_labels_mat = ch_labels_mat.tolist()
            if not isinstance(ch_labels_mat, list): # If it's a single string, make it a list
                 ch_labels_mat = [ch_labels_mat]
        else:
            print("  'chanlocs' has an unexpected type or structure. Cannot automatically extract labels.")
        
        print(f"\nExtracted Channel Labels ('ch_labels_mat') ({len(ch_labels_mat)} channels):")
        print(ch_labels_mat) # This is the list you need to examine!

    else:
        print(" 'chanlocs' field not found in 'eeg_struct'.")


    # --- Step 4: Extract Event Information ---
    print("\n--- Inspecting EEG.event ---")
    events_eeglab = eeg_struct.get('event', None)
    if events_eeglab is None:
        print(" 'event' field not found in 'eeg_struct'.")
    elif isinstance(events_eeglab, list) and len(events_eeglab) > 0:
        print(f" 'event' field is a list with {len(events_eeglab)} events. Inspecting first 5:")
        for i, event in enumerate(events_eeglab[:5]):
            print(f"  Event {i+1}: Type='{event.get('type', 'N/A')}', Latency='{event.get('latency', 'N/A')}'")
    else:
        print(f" 'event' field has unexpected type: {type(events_eeglab)}")


    # --- FINAL VERIFICATION STEPS ---
    print("\n--- Summary for config.py Update ---")
    print("\nBased on the 'Extracted Channel Labels' list above:")
    print("  1. Find your VEOG channel name (e.g., 'VEOG', 'EOG_V').")
    print("  2. Find your Accelerometer channel names (e.g., ['X_Accel', 'Y_Accel', 'Z_Accel'] or ['ACCEL_X', 'ACCEL_Y', 'ACCEL_Z']).")
    print("  3. Find your Stimulus channel name (e.g., 'STI 014', 'Trigger').")
    print("\nThen, update these lines in 'src/eeg/config.py' accordingly:")
    print(f"VEOG_CHANNEL_NAME = 'YOUR_VEOG_NAME_HERE'")
    print(f"ACCEL_CHANNEL_NAMES = ['YOUR_ACCEL_X_NAME', 'YOUR_ACCEL_Y_NAME', 'YOUR_ACCEL_Z_NAME']")
    print(f"STIM_CHANNEL_NAME = 'YOUR_STIM_NAME_HERE'")
    print("\nAlso, confirm your raw MATLAB filenames: 'mat_fname_base' in '00_convert_mat_to_fif.py' should match.")
    print("And remember to set OVERWRITE_PROCESSED_FILES = True in config.py for testing.")


except FileNotFoundError:
    print(f"Error: MATLAB file not found at {mat_fname_full}. Please check path and name.")
except Exception as e:
    print(f"An error occurred during interactive loading: {e}")
    import traceback
    traceback.print_exc()


Successfully loaded /Users/patriciawatanabe/Projects/NTUT25_Software/data/matlab_file/804_1_PD_REST.mat

--- Found 'EEG' key at top level. Proceeding with eeg_struct = mat_data['EEG']. ---

Type of 'eeg_struct': <class 'dict'>
Keys within 'eeg_struct': dict_keys(['setname', 'filename', 'filepath', 'subject', 'group', 'condition', 'session', 'comments', 'nbchan', 'trials', 'pnts', 'srate', 'xmin', 'xmax', 'times', 'data', 'icaact', 'icawinv', 'icasphere', 'icaweights', 'icachansind', 'chanlocs', 'urchanlocs', 'chaninfo', 'ref', 'event', 'urevent', 'eventdescription', 'epoch', 'epochdescription', 'reject', 'stats', 'specdata', 'specicaact', 'splinefile', 'icasplinefile', 'dipfit', 'history', 'saved', 'etc'])

--- Inspecting EEG.data and EEG.srate ---
 'eeg_struct['data']' shape: (67, 96501)
 'eeg_struct['data']' first 5 samples of first channel: [-11679.736 -11683.691 -11685.645 -11693.604 -11691.211]
 'eeg_struct['data']' type: <class 'numpy.ndarray'>
 'eeg_struct['srate']': 500.0
 'eeg